In [1]:
import sys
sys.path.append('../../../src/')

import os
from pathlib import Path

from utils import parse, AddExtraColumns

# Plotting libraries
import datetime
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

color1='#d95f02'
color2='#1b9e77'
color3='#7570b3'
odor_list_color = [color1, color2, color3]

pdf_path = r'Z:\scratch\vr-foraging\sessions'
base_path = 'Z:/scratch/vr-foraging/data/'
foraging_figures = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\VR foraging\results'

In [ ]:
date = datetime.date.today()
date_string = "06/21/2024"
date = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()
mouse_list = ["715866", "713578", "707349","715865","715869","713545","715867","715870","716455","716456","716457", "716458"]

In [3]:
cumulative_df = pd.DataFrame(columns=['mouse', 'session', 'total_sites', 'rewarded_stops', 'unrewarded_stops', 'water', 'distance', 'session_n'])
cumulative_df_trials = pd.DataFrame()
for mouse in mouse_list:
    print(mouse)
    session_n = 0
    session_found = False

    directory = os.path.join(base_path, mouse)
    files = os.listdir(os.path.join(base_path, mouse))
    
    sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)), reverse=True)
    
    for file_name in sorted_files:
            
        # Recover data streams
        session_path = os.path.join(base_path, mouse, file_name)
        session_path = Path(session_path)
        session = file_name[:8]
        
        data = parse.load_session_data(session_path)

        # Parse data
        reward_sites, active_site, config = parse.parse_dataframe(data)
        reward_sites = AddExtraColumns(reward_sites, active_site, run_on_init=True).reward_sites
        all_epochs = AddExtraColumns(reward_sites, active_site, run_on_init=True).total_epochs
        
        encoder_data = parse.ContinuousData(data).encoder_data
        session_n += 1
        
        reward_sites['odor_duration'] = reward_sites['odor_offset'] - reward_sites['odor_onset']
        
        # Remove segments where the mouse was disengaged
        last_engaged_patch = reward_sites['active_patch'][reward_sites['skipped_count'] >= 7].min()
        if pd.isna(last_engaged_patch):
            last_engaged_patch = reward_sites['active_patch'].max()
        reward_sites['disengaged'] = reward_sites['active_patch'] >= last_engaged_patch      
        
        # Summary of different relevants aspects -------------------------------------------------
        collected_df = reward_sites['collected'].sum()

        unrewarded_stops = reward_sites.loc[reward_sites.reward_delivered==0]['collected'].count()
        rewarded_stops = reward_sites.loc[reward_sites.reward_delivered==1]['collected'].count()
        water_collected = reward_sites.loc[(reward_sites['reward_delivered']==1)]['collected'].sum()
        total_stops = reward_sites.loc[(reward_sites['has_choice']==True)]['reward_available'].count()
        stopped_df = reward_sites.loc[(reward_sites['has_choice']==True)].groupby(['collected','odor_label'])[['reward_delivered']].sum().reset_index()

        # Rewarded stops / total available rewarded stops
        optimality = rewarded_stops / reward_sites[reward_sites.reward_available != 0]['reward_delivered'].count()

        # Rewarded stops / Stops
        efficiency = rewarded_stops / reward_sites.reward_delivered.sum()

        print('Total sites: ' ,len(reward_sites), ' | ', 'Total rewarded stops: ',rewarded_stops, '(',  np.round((rewarded_stops/total_stops)*100,2),'%) | ', 
            'Total unrewarded stops: ',unrewarded_stops,'(',  np.round((unrewarded_stops/total_stops)*100,2),'%) | ','Water consumed: ', water_collected, 'ul')

        if 'startPosition' in active_site.columns:
            distance = np.round(active_site.startPosition.max()/100,2)
        else:
            distance = np.round(active_site.start_position.max()/100,2)
        print('Total travelled m: ', np.round(active_site.start_position.max()/100,2))

        for odor_label in reward_sites.odor_label.unique():
            values = reward_sites.loc[(reward_sites['odor_label']==odor_label)&(reward_sites['reward_delivered']==1)]['collected'].sum()
            print(f'{odor_label} {values} ul')
        
        cumulative_df_trials['mouse'] = mouse
        cumulative_df_trials['session'] = session
        cumulative_df_trials['session_n'] = session_n
        cumulative_df_trials = pd.concat([cumulative_df_trials, reward_sites], ignore_index=True)

        # if session not in cumulative_df.session.unique() and :
        new_row = {'mouse':mouse, 'session': session, 'total_sites':len(reward_sites), 'rewarded_stops':rewarded_stops, 'unrewarded_stops':unrewarded_stops, 
                    'water':water_collected, 'distance_m':distance, 'session_n':session_n}
        cumulative_df.loc[len(cumulative_df)] = new_row
          
        break


715866
Total sites:  451  |  Total rewarded stops:  216 ( 79.7 %) |  Total unrewarded stops:  235 ( 86.72 %) |  Water consumed:  1080.0 ul
Total travelled m:  749.87
2,3-Butanedione 575.0 ul
Methyl Acetate 505.0 ul
Fenchone 0.0 ul
713578
Total sites:  431  |  Total rewarded stops:  192 ( 70.85 %) |  Total unrewarded stops:  239 ( 88.19 %) |  Water consumed:  960.0 ul
Total travelled m:  673.79
2,3-Butanedione 740.0 ul
Methyl Acetate 220.0 ul
Fenchone 0.0 ul
707349
Total sites:  498  |  Total rewarded stops:  231 ( 53.47 %) |  Total unrewarded stops:  267 ( 61.81 %) |  Water consumed:  1155.0 ul
Total travelled m:  566.47
2,3-Butanedione 520.0 ul
Fenchone 0.0 ul
Methyl Acetate 635.0 ul
715865
Total sites:  426  |  Total rewarded stops:  237 ( 69.1 %) |  Total unrewarded stops:  189 ( 55.1 %) |  Water consumed:  1185.0 ul
Total travelled m:  522.86
2,3-Butanedione 840.0 ul
Methyl Acetate 345.0 ul
Fenchone 0.0 ul
715869
Total sites:  520  |  Total rewarded stops:  262 ( 61.65 %) |  Total 

In [9]:
reward_sites.collected.sum() / (reward_sites.loc[reward_sites.disengaged==False].index[-1] - active_site.index[0])

0.5736027127622341

In [ ]:
all_epochs = pd.concat([reward_sites, active_site.loc[active_site.label != 'RewardSite']])
all_epochs.sort_index(inplace=True)

In [ ]:
active_patch = -1
total_sites = -1
for i, row in all_epochs.iterrows():
    if row['label'] == 'InterPatch':
        active_patch += 1
    if row['label'] == 'InterSite':
        total_sites += 1
    all_epochs.at[i, 'active_patch'] = active_patch
    all_epochs.at[i, 'total_sites'] = total_sites
all_epochs['total_sites'] = np.where(all_epochs['total_sites'] == -1, 0, all_epochs['total_sites'])